In [ ]:
import pandas
import datetime
import time
# import the dataset creation module
import os
import sys
module_path = os.path.abspath(os.path.join('../data_pipeline/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import create_onset_data
import omn_utils
import batch_utils

In [ ]:
# dataObj = create_onset_data.OnsetData()

In [ ]:
# print len(dataObj.polarDateList), len(dataObj.imageDateList)

In [ ]:
# x = time.time()
# finDF = dataObj.create_output_bins()
# y = time.time()
# print y-x

In [ ]:
# finDF.head()

In [ ]:
# finDF = pandas.read_feather("../data/binned_data.feather")
# finDF.head()

In [ ]:
omn_dbdir = "/home/bharat/Documents/data/ss_onset_dataset/data/sqlite3/"
omn_db_name = "omni_sw_imf.sqlite"
omn_table_name = "imf_sw"
omn_train = True
omn_norm_param_file = omn_dbdir + "omn_mean_std.npy"
# start_date = finDF["date"].min()
# end_date = finDF["date"].max()
# print start_date, end_date

In [ ]:
# omnObj = omn_utils.OmnData(start_date, end_date, omn_dbdir,\
#                            omn_db_name, omn_table_name,\
#                            omn_train, omn_norm_param_file)
# omnObj.omnDF.head()

In [ ]:
batchObj = batch_utils.DataUtils(omn_dbdir,\
                           omn_db_name, omn_table_name,\
                           omn_train, omn_norm_param_file)

In [ ]:
b1 = batchObj.batchDict[0]
outOn = batchObj.onset_from_batch(b1)
outOmn = batchObj.omn_from_batch(b1)

In [ ]:
print outOmn.shape